<a href="https://colab.research.google.com/github/xandreiAThome/machine-translation-nlp1k/blob/main/nmt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Neural Machine Translation

## Preprocess

Load the aligned verses from the tsv, clean the string from any non alphabetic characters. Remove any verses that have no verse for either of the two language, and use the class from the datasets library to structure the data and be ready for training.

In [39]:
import regex as re

def clean_string(input_string):
    cleaned = re.sub(r"[^\p{L}\s]", "", input_string.strip().lower())
    return cleaned

def process(example):
    src = example["src"].strip()
    tgt = example["tgt"].strip()

    # skip invalid pairs
    if src.lower() == "<no verse>" or tgt.lower() == "<no verse>":
        return {"src": None, "tgt": None}

    return {
        "src": clean_string(src),
        "tgt": clean_string(tgt),
    }

In [40]:
src_lang = "Bikolano"
target_lang = "Tagalog"

In [41]:
from datasets import load_dataset

dataset = load_dataset(
    "csv",
    data_files="data/dataset/Bikolano_Tagalog_Parallel.tsv",
    delimiter="\t",
)

dataset = dataset["train"].select_columns([src_lang, target_lang])
dataset = dataset.rename_columns({src_lang: "src", target_lang: "tgt"})

# Get initial dataset length
initial_dataset_length = len(dataset)

dataset = dataset.map(process)

# remove rows with None (invalid)
dataset = dataset.filter(lambda x: x["src"] is not None and x["tgt"] is not None)

# Calculate skipped verses
skipped = initial_dataset_length - len(dataset)
print(f"skipped verses: {skipped}")

skipped verses: 27


Lets look at the first 5 aligned verses

In [42]:
display(dataset[:5])

{'src': ['si adan iyo an ama ni set asin si set iyo an ama ni enos na ama ni kenan',
  'si kenan iyo an ama ni mahalalel na ama ni jared',
  'si jared iyo an ama ni enoc na ama ni metusela si metusela iyo an ama ni lamec',
  'na iyo an ama ni noe si noe nagkaigwa nin tolong aking lalaki na iyo si sem ham asin si jafet',
  'an mga aking lalaki ni jafet iyo si gomer magog madai javan tubal mesec asin tiras'],
 'tgt': ['sina adan set enos',
  'kenan mahalalel jared',
  'enoc matusalem lamec',
  'noe sem ham at jafet',
  'ang mga anak ni jafet ay sina gomer magog madai javan tubal meshec at tiras']}

## Setting up Trainer
We will use facebook's No Language Left Behind Model as the base model to fine tune using our dataset. It is performant even on low resource languages thats why our group decided to use it.

In [44]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "facebook/nllb-200-distilled-600M"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [45]:
def tokenize(batch):
    model_inputs = tokenizer(batch["src"], truncation=True, max_length=128)
    labels = tokenizer(batch["tgt"], truncation=True, max_length=128).input_ids
    model_inputs["labels"] = labels
    return model_inputs

tokenized_dataset = dataset.map(tokenize, batched=True)

Map:   0%|          | 0/33873 [00:00<?, ? examples/s]

Let us split the training data to also have a dataset for evaluation after training.

In [48]:
split = tokenized_dataset.train_test_split(test_size=0.1)
train_data = split["train"]
eval_data = split["test"]

In [57]:
from transformers import Seq2SeqTrainingArguments,

training_args = Seq2SeqTrainingArguments(
    output_dir="./nllb-bcl-tgl",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    learning_rate=5e-5,
    num_train_epochs=8,
    eval_strategy="epoch", # Changed from evaluate_during_training
    save_strategy="epoch",
    logging_steps=50,
    fp16=True,
    gradient_accumulation_steps=2,  # effective batch size = 8
    weight_decay=0.01,
    predict_with_generate=True,
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=eval_data,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

/tmp/ipython-input-2212020708.py:20: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [58]:
trainer.train()
trainer.save_model("./nllb-bcl-tgl")
tokenizer.save_pretrained("./nllb-bcl-tgl")


/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: xandrei37 (xandrei37-de-la-salle-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


KeyboardInterrupt: 